In [4]:
import pandas as pd
import numpy as np


In [5]:
def calculate_distance(df, name1, name2, x_col, y_col, z_col, print_values=True):
    point1 = df[df["Mic_Index"] == name1][[x_col, y_col, z_col]].iloc[0]
    point2 = df[df["Mic_Index"] == name2][[x_col, y_col, z_col]].iloc[0]
    
    # euklidische Distanz berechnen
    dist = np.sqrt((point2[x_col] - point1[x_col])**2 + 
                   (point2[y_col] - point1[y_col])**2 + 
                   (point2[z_col] - point1[z_col])**2)
    
    if print_values == True:
        print(name1)
        print(x_col, ":", round(point1[x_col], 3))
        print(y_col, ":", round(point1[y_col], 3))
        print(z_col, ":", round(point1[z_col], 3))
        print("")
        print(name2)
        print(x_col, ":", round(point2[x_col], 3))
        print(y_col, ":", round(point2[y_col], 3))
        print(z_col, ":", round(point2[z_col], 3))

        print("distance = ", round(dist, 3))


    return dist

In [6]:

file_path = r"data\side_postions_measured.xlsx"
df = pd.read_excel(file_path, engine='openpyxl')

# cut empty rows
df = df.head(64)

In [7]:
df_3d = df.copy()


# all distances in meteres
# grid = innenseite vom gitter an den Seiten
# frame = unterseite vom Aluprofil 

MicOffsetInside = 0.028 # wie weit die mics vom grid nach innen reichen
MicOffsetInsideTop = 0.013 # wie weit die mics vom frame nach innen reichen

side_grid_width = 1.378 #measured width of side grid
top_grid_width_Y = 1.23 # measured width of top frame ( 2D-Y direction )
top_grid_width_X = 1.46 # measured width of top frame ( 2D-X direction )

TopFrameYOffset = 0.069 # Y distance between front plane and edge of top grid (Gitter zu Aluprofilkante)

# X distance between edge of top frame and left / right grid
XSideOffsetLeft = 0.01
XSideOffsetRight = 0.01

# Y  distance between front grid and edge of left/right grid
YSideOffsetLeft = 0.088
YSideOffsetRight = 0.085

# distance between top frame and lower edge of side grids (Z = 0)
TopOffset = 1.884

x_2d = "X measured"
y_2d = "Y measured"

# translate 2d coordinates to 3d coordinates
# for left plane
df_3d.loc[df_3d["Plane"] == "left", "X"] = 0 + MicOffsetInside
df_3d.loc[df_3d["Plane"] == "left", "Y"] = -df[x_2d] + side_grid_width + YSideOffsetLeft
df_3d.loc[df_3d["Plane"] == "left", "Z"] = df[y_2d]

# for front plane
df_3d.loc[df_3d["Plane"] == "front", "X"] = df[x_2d] + XSideOffsetLeft
df_3d.loc[df_3d["Plane"] == "front", "Y"] = 0 + MicOffsetInside
df_3d.loc[df_3d["Plane"] == "front", "Z"] = df[y_2d]

# for right plane
df_3d.loc[df_3d["Plane"] == "right", "X"] = side_grid_width + XSideOffsetLeft + XSideOffsetRight - MicOffsetInside
df_3d.loc[df_3d["Plane"] == "right", "Y"] = df[x_2d]  + YSideOffsetRight
df_3d.loc[df_3d["Plane"] == "right", "Z"] = df[y_2d]

# for top plane
df_3d.loc[df_3d["Plane"] == "top", "X"] = -df[x_2d] + top_grid_width_X + XSideOffsetLeft
df_3d.loc[df_3d["Plane"] == "top", "Y"] = -df[y_2d] + top_grid_width_Y + TopFrameYOffset
df_3d.loc[df_3d["Plane"] == "top", "Z"] = TopOffset - MicOffsetInsideTop


columns_order = [
    "Unnamed: 0",
    "Mic_Index",
    "Plane",
    "X",
    "Y",
    "Z",
    "X measured",
    "Y measured",
]
df_3d = df_3d[columns_order]


# Ergebnis anzeigen
print(df_3d)


    Unnamed: 0 Mic_Index  Plane      X      Y      Z  X measured  Y measured
0          0.0        B1  right  1.370  0.734  0.020       0.649       0.020
1          1.0        B2  right  1.370  0.987  0.266       0.902       0.266
2          2.0        B3  right  1.370  0.336  0.287       0.251       0.287
3          3.0        B4  right  1.370  1.398  0.380       1.313       0.380
4          4.0        B5  right  1.370  0.647  0.425       0.562       0.425
..         ...       ...    ...    ...    ...    ...         ...         ...
59        59.0        A9    top  1.405  0.521  1.871       0.065       0.778
60        60.0       A10    top  0.680  0.393  1.871       0.790       0.906
61        61.0       A11    top  0.985  0.155  1.871       0.485       1.144
62        62.0       A12    top  0.332  0.129  1.871       1.138       1.170
63        63.0       A13    top  1.272  0.117  1.871       0.198       1.182

[64 rows x 8 columns]


In [8]:
_ = calculate_distance(df_3d, "A12", "D16", x_col="X", y_col="Y", z_col="Z")

A12
X : 0.332
Y : 0.129
Z : 1.871

D16
X : 0.028
Y : 0.161
Z : 1.511
distance =  0.472


In [9]:
# fancy interactiver plot 
# damit kann man die namen der mikrofone sehr gut sehen


import plotly.express as px


fig = px.scatter_3d(df_3d, x='X', y='Y', z='Z', text='Mic_Index', title='3D Scatter Plot', color='Plane', width=1000, height=1000)

# Plot anzeigen
fig.show()

In [10]:
_ = calculate_distance(df_3d, "D13", "D16", x_col="X", y_col="Y", z_col="Z") # 1,26 m
_ = calculate_distance(df_3d, "B17", "C16", x_col="X", y_col="Y", z_col="Z") # 0,67 m
_ = calculate_distance(df_3d, "C10", "D10", x_col="X", y_col="Y", z_col="Z") # 1,175 m
_ = calculate_distance(df_3d, "C2", "B15", x_col="X", y_col="Y", z_col="Z") # 1,65 m
_ = calculate_distance(df_3d, "A7", "B6", x_col="X", y_col="Y", z_col="Z") # 1,416 m
_ = calculate_distance(df_3d, "A6", "D17", x_col="X", y_col="Y", z_col="Z") # 0,40 m
_ = calculate_distance(df_3d, "A10", "C5", x_col="X", y_col="Y", z_col="Z") # 1,485 m
_ = calculate_distance(df_3d, "C4", "D3", x_col="X", y_col="Y", z_col="Z") # 1,77 m
_ = calculate_distance(df_3d, "D1", "B13", x_col="X", y_col="Y", z_col="Z") #1,975 m
_ = calculate_distance(df_3d, "A5", "B8", x_col="X", y_col="Y", z_col="Z") # 1,301 m
_ = calculate_distance(df_3d, "D7", "B8", x_col="X", y_col="Y", z_col="Z") #1,385 m
_ = calculate_distance(df_3d, "D13", "B16", x_col="X", y_col="Y", z_col="Z") # 1,407 m
_ = calculate_distance(df_3d, "A1", "C16", x_col="X", y_col="Y", z_col="Z") # 1,485 m
_ = calculate_distance(df_3d, "A4", "D5", x_col="X", y_col="Y", z_col="Z") # 1,963

D13
X : 0.028
Y : 1.392
Z : 1.26

D16
X : 0.028
Y : 0.161
Z : 1.511
distance =  1.256
B17
X : 1.37
Y : 0.674
Z : 1.689

C16
X : 1.313
Y : 0.028
Z : 1.508
distance =  0.673
C10
X : 0.299
Y : 0.028
Z : 0.99

D10
X : 0.028
Y : 1.177
Z : 0.99
distance =  1.181
C2
X : 0.913
Y : 0.028
Z : 0.267

B15
X : 1.37
Y : 1.044
Z : 1.485
distance =  1.651
A7
X : 0.755
Y : 0.81
Z : 1.871

B6
X : 1.37
Y : 0.989
Z : 0.629
distance =  1.397
A6
X : 0.411
Y : 0.856
Z : 1.871

D17
X : 0.028
Y : 0.877
Z : 1.69
distance =  0.424
A10
X : 0.68
Y : 0.393
Z : 1.871

C5
X : 0.571
Y : 0.028
Z : 0.425
distance =  1.495
C4
X : 1.322
Y : 0.028
Z : 0.38

D3
X : 0.028
Y : 1.215
Z : 0.289
distance =  1.758
D1
X : 0.028
Y : 0.817
Z : 0.023

B13
X : 1.37
Y : 0.157
Z : 1.257
distance =  1.939
A5
X : 1.028
Y : 0.857
Z : 1.871

B8
X : 1.37
Y : 1.315
Z : 0.718
distance =  1.287
D7
X : 0.028
Y : 1.331
Z : 0.677

B8
X : 1.37
Y : 1.315
Z : 0.718
distance =  1.343
D13
X : 0.028
Y : 1.392
Z : 1.26

B16
X : 1.37
Y : 1.392
Z : 1.51
di